![Header](https://github.com/VarunBurde/Prague_ml/blob/main/visulization_data/header.png?raw=true)

# Instant Neural Graphics Primitives (Instant-NGP)

## Introduction

![Instant-NGP Example Renders](https://raw.githubusercontent.com/NVlabs/instant-ngp/master/docs/assets_readme/fox.gif)

Instant NGP is a neural network architecture developed by NVIDIA Research that enables high-quality 3D reconstruction and rendering at unprecedented speeds. It uses a multi-resolution hash encoding technique to drastically accelerate neural graphics applications such as NeRF (Neural Radiance Fields), allowing models to be trained in seconds or minutes rather than hours or days.

**Research Paper:** [Müller et al., "Instant Neural Graphics Primitives with a Multiresolution Hash Encoding", ACM Transactions on Graphics (SIGGRAPH), 2022](https://nvlabs.github.io/instant-ngp/assets/mueller2022instant.pdf)

**Key features of Instant-NGP:**
- Ultra-fast training and inference times (15-30x faster than previous approaches)
- High-quality 3D reconstructions from 2D images
- Support for various applications including NeRF, signed distance functions, and volume rendering
- Achieves state-of-the-art quality with significantly reduced computational requirements

This notebook guides you through the process of downloading, setting up, and running Instant-NGP to create your own 3D models from 2D images.

**Total time to run this notebook:** Approximately 15 minutes

![NGP Hash Grid](https://github.com/VarunBurde/Prague_ml/blob/main/visulization_data/hash_grid.png?raw=true)

## Step 1: Download the Instant-NGP Source Code

First, we'll download the Instant-NGP package using gdown from Google Drive to reduce the time for building. This package contains the pre-built Instant-NGP software optimized for GPU acceleration.

The official repository is available at [https://github.com/NVlabs/instant-ngp](https://github.com/NVlabs/instant-ngp) if you want to learn more about the implementation.

In [1]:
import gdown

url = 'https://drive.google.com/file/d/1WwpOoQ-6YY0Qd2SHVXNdAtatg51wjEnf/view?usp=drive_link'
output_path = 'instant-ngp.zip'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1WwpOoQ-6YY0Qd2SHVXNdAtatg51wjEnf
From (redirected): https://drive.google.com/uc?id=1WwpOoQ-6YY0Qd2SHVXNdAtatg51wjEnf&confirm=t&uuid=98d5e540-dc03-4aa0-a042-05963266eed4
To: /content/instant-ngp.zip
100%|██████████| 1.46G/1.46G [00:18<00:00, 77.2MB/s]


'instant-ngp.zip'

In [2]:
!unzip /content/instant-ngp.zip -d /

Streaming output truncated to the last 5000 lines.
  inflating: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/structcutlass_1_1minus_3_01Array_3_01T_00_01N_01_4_01_4.html  
  inflating: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/structcutlass_1_1reference_1_1host_1_1detail_1_1TensorForEachHelper.html  
  inflating: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/simd_8h.html  
  inflating: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/functions_vars_v.html  
 extracting: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/inherit_graph_529.md5  
  inflating: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/classcutlass_1_1transform_1_1threadblock_1_1PredicatedTileAccessIterator2dThreadTile_3_01Shape__ad17304f9466e09edfd94345da01b287.html  
  inflating: /content/instant-ngp/dependencies/tiny-cuda-nn/dependencies/cutlass/docs/classcutla

## Step 2: Download the Prepared Dataset

For this demonstration, we'll use a pre-prepared dataset containing multiple images of an object captured from different viewpoints. The dataset includes camera parameters required for 3D reconstruction.

The dataset is structured and converted to instant ngp format from the previous notebook you have tried:
- **Images**: Multiple views of the subject from different angles
- **Camera parameters**: Information about camera position and orientation for each image
- **Calibration data**: Lens and sensor parameters needed for accurate 3D reconstruction

In [3]:
url = "https://drive.google.com/file/d/10XpQY8pdQKEv3VATQyte-Os01mrTWDju/view?usp=drive_link"
output_path = 'object_dataset.zip'
gdown.download(url, output_path, quiet=False,fuzzy=True)


Downloading...
From (original): https://drive.google.com/uc?id=10XpQY8pdQKEv3VATQyte-Os01mrTWDju
From (redirected): https://drive.google.com/uc?id=10XpQY8pdQKEv3VATQyte-Os01mrTWDju&confirm=t&uuid=4818d1ee-130a-45cc-9df9-44263de6d913
To: /content/object_dataset.zip
100%|██████████| 172M/172M [00:03<00:00, 45.4MB/s]


'object_dataset.zip'

In [ ]:
!unzip /content/object_dataset.zip -d /content/

Archive:  /content/object_dataset.zip
   creating: /content/object_dataset/
   creating: /content/object_dataset/rgb/
  inflating: /content/object_dataset/rgb/000146.png  
  inflating: /content/object_dataset/rgb/000032.png  
  inflating: /content/object_dataset/rgb/000026.png  
  inflating: /content/object_dataset/rgb/000027.png  
  inflating: /content/object_dataset/rgb/000033.png  
  inflating: /content/object_dataset/rgb/000147.png  
  inflating: /content/object_dataset/rgb/000145.png  
  inflating: /content/object_dataset/rgb/000019.png  
  inflating: /content/object_dataset/rgb/000025.png  
  inflating: /content/object_dataset/rgb/000031.png  
  inflating: /content/object_dataset/rgb/000030.png  
  inflating: /content/object_dataset/rgb/000024.png  
  inflating: /content/object_dataset/rgb/000018.png  
  inflating: /content/object_dataset/rgb/000144.png  
  inflating: /content/object_dataset/rgb/000150.png  
  inflating: /content/object_dataset/rgb/000140.png  
  inflating: /cont

## Step 3: Install System Dependencies

Next, we need to install several system dependencies required for building and running Instant-NGP. These libraries provide essential functionality for 3D graphics processing, multi-threading, and GPU acceleration.

**Key dependencies include:**
- **build-essential**: Contains reference libraries and tools for building software
- **libopenexr-dev**: Library for the OpenEXR image format (high dynamic range imaging)
- **libglfw3-dev & libglew-dev**: Libraries for OpenGL development and GPU acceleration
- **libomp-dev**: Library for OpenMP multi-threading support to utilize all CPU cores
- **Other dependencies**: Support libraries for GUI elements (though we'll run in headless mode)


In [5]:
!sudo apt-get install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev \
                     libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libopenexr-dev is already the newest version (2.5.7-1).
libopenexr-dev set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.12).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following additional packages will be installed:
  libegl-dev libegl1-mesa-dev libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglfw3 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev
  libglx-dev libomp-14-dev libomp5-14 libopengl-dev libvulkan-dev libvulkan1
  libwayland-bin libwayland-dev libxfixes-dev libxrandr-dev
  mesa-vulkan-drivers python3-setuptools python3-wheel
Suggested packages:
  libosmesa6 libglfw3-doc libomp-14-doc libwayland-doc python-setuptools-doc
The following NEW packages will be installed:
  libegl-dev libegl1-mesa-dev libgl-dev libgl1-m

## Step 4: Install Python Dependencies

Now we'll install the required Python packages needed by Instant-NGP. These include libraries for:
- Image processing and manipulation
- Mathematical operations and linear algebra
- Deep learning frameworks
- CUDA integration for GPU acceleration

The requirements.txt file contains all the necessary Python dependencies with specific version requirements to ensure compatibility.

In [6]:
%cd /content/instant-ngp/
!pip install -r requirements.txt


/content/instant-ngp
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 21.2 MB/s eta 0:00:00
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12077 sha256=0c540dca350442ea288931d060ce22a1d2d677c8faa3608be71c942aaf749ff0
  Stored in directory: /root/.cache/pip/wheels/f2/cb/58/77f1f0b3f5fa6eb3335cb0cb73de4d4bef93489dc3a3373ae8
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62511 sha256=4714954dad06ea8b3ea55e505bda60cb2b6d71d867c29a4051bcf3ddc24d6040
  Stored in directory: /root/.cache/pip/wheels/6c/c7/01/d277a7a673ffc81a1e71e816c2e5bc5f1602f9e8192c25d21c
Successfully built commentjson lark-parser


## Visulize some of the training images 

## Step 5: Train and Export a 3D Model

Now we'll train a Neural Radiance Field (NeRF) using Instant-NGP's accelerated architecture. This process consists of several phases:

**How the process works:**

1. **Input processing**: The system analyzes all images and camera parameters from the dataset
2. **Hash encoding**: Instant-NGP uses a multi-resolution hash table to efficiently encode the 3D space
3. **Neural network training**: A compact MLP network learns to predict color and density for any 3D point
4. **Volume rendering**: The trained network renders novel viewpoints through ray marching
5. **Mesh extraction**: The final 3D model is extracted using techniques like Marching Cubes

The `--n_steps 30000` parameter controls the number of training iterations. The `--save_mesh output.ply` option exports the resulting 3D model as a PLY file that can be imported into 3D software.

In [ ]:
!python scripts/run.py --scene /content/object_dataset/ --n_steps 30000 --train --save_mesh output.ply

15:48:31 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
15:48:31 INFO     Loading NeRF dataset from
15:48:31 INFO       /content/object_dataset/transforms.json
15:48:31 PROGRESS []   0% (  0/149)  0s/inf15:48:31 PROGRESS []   1% (  1/149) 0s/5s15:48:31 PROGRESS []   1% (  2/149) 0s/3s15:48:31 PROGRESS []   2% (  3/149) 0s/3s15:48:31 PROGRESS []   3% (  4/149) 0s/4s15:48:31 PROGRESS []   3% (  5/149) 0s/4s15:48:31 PROGRESS []   4% (  6/149) 0s/3s15:48:31 PROGRESS []   5% (  7/149) 0s/4s15:48:32 PROGRESS []   5% (  8/149) 0s/4s15:48:32 PROGRESS []   6% (  9/149) 0s/4s15:48:32 PROGRESS []   7% ( 10/149) 0s/4s15:48:32 PROGRESS []   7% ( 11/149) 0s/4s15:48:32 PROGRESS []   8% ( 12/149) 0s/4s15:48:32 PROGRESS []   9% ( 13/149) 0s/4s15:48:32 PROGRESS []   9% ( 14/149) 0s/4s15:48:32 PROGRESS []  10% ( 15/149) 0s/4s15:48:32 PROGRESS []  11% ( 16/149) 0s/4s15:48:32 PROGRESS []  11% ( 17/149) 0s/4s15:48:32 PROGRESS []  12% ( 18/149) 0s/4s15:48:32 PROGRESS []  13% ( 19/149) 0s/4s1

## Results and Next Steps

After successfully training the model, you'll have a high-quality 3D reconstruction of your object that can be viewed or edited in 3D software.

![Example Result](https://github.com/VarunBurde/Prague_ml/blob/main/visulization_data/ngp_visulization.png?raw=true)

### What you can do next:

- **Download the output.ply mesh file** to view the 3D model in software like [Blender](https://www.blender.org/) or [MeshLab](https://www.meshlab.net/)
- **Experiment with different datasets** by replacing the `--scene` parameter with your own image collections
- **Adjust quality settings** with parameters like:
  - `--n_steps`: Increase for better quality, decrease for faster training
  - `--resolution`: Control output resolution and detail level
  - `--exposure`: Fine-tune the brightness of the model
- **Create animations** by using the `--save_snapshot` option to save the trained model state

### Further Reading:

- [Instant NGP GitHub Repository](https://github.com/NVlabs/instant-ngp)
- [Original NeRF Paper](https://www.matthewtancik.com/nerf)
- [Müller et al., "Instant Neural Graphics Primitives with a Multiresolution Hash Encoding"](https://nvlabs.github.io/instant-ngp/)


![Footer](https://github.com/VarunBurde/Prague_ml/blob/main/visulization_data/footer.png?raw=true)